In [ ]:
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-hlb6w10e
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-hlb6w10e
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=aec2415c4240432d2f4a2cd0c8b25dfa4105f4c8615b8f3d65b46c03bb214760
  Stored in directory: /tmp/pip-ephem-wheel-cache-1bxev8vp/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
import torch
import skimage.io as io
import clip
from PIL import Image
import pickle
import json
import os
from tqdm import tqdm


def main():
    device = torch.device('cuda:0')
    out_path = "/content/drive/MyDrive/Colab Notebooks/Dataset 1/split_train.pkl"
    clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
    with open('/content/drive/MyDrive/Colab Notebooks/Dataset 1/flickr_caption.json', 'r') as f:
        data = json.load(f)
    print("%0d captions loaded from json " % len(data))
    all_embeddings = []
    all_captions = []
    for i in tqdm(range(len(data))):
        d = data[i]
        img_id = d["image_name"]
        filename = f"/content/drive/MyDrive/Colab Notebooks/Dataset 1/Dataset Splitted/train/{int(img_id)}.jpg"
        if not os.path.isfile(filename):
            filename = f"/content/drive/MyDrive/Colab Notebooks/Dataset 1/Dataset Splitted/val/{int(img_id)}.jpg"
            if not os.path.isfile(filename):
                filename = f"/content/drive/MyDrive/Colab Notebooks/Dataset 1/Dataset Splitted/test/{int(img_id)}.jpg"
        image = io.imread(filename)
        image = preprocess(Image.fromarray(image)).unsqueeze(0).to(device)
        with torch.no_grad():
            prefix = clip_model.encode_image(image).cpu()
        d["clip_embedding"] = i
        all_embeddings.append(prefix)
        all_captions.append(d)
        if (i + 1) % 10000 == 0:
            with open(out_path, 'wb') as f:
                pickle.dump({"clip_embedding": torch.cat(all_embeddings, dim=0), "captions": all_captions}, f)

    with open(out_path, 'wb') as f:
        pickle.dump({"clip_embedding": torch.cat(all_embeddings, dim=0), "captions": all_captions}, f)

    print('Done')
    print("%0d embeddings saved " % len(all_embeddings))
    return 0


if __name__ == '__main__':
    exit(main())

100%|███████████████████████████████████████| 338M/338M [00:31<00:00, 11.2MiB/s]


158915 captions loaded from json 


100%|██████████| 158915/158915 [3:01:57<00:00, 14.56it/s]


Done
158915 embeddings saved 
